In [ ]:
%load_ext autoreload
%autoreload 2

import json
import numpy as np
from idqn.utils.confidence_interval import confidence_interval

experiment_name = "q35_N50000_b100_b20_f4_t1_lr4_5_t"
list_bellman_iterations_scope = [1, 2, 4, 5, 10, 20]
seeds = range(1, 2)

p = json.load(open(f"old_figures/{experiment_name}/parameters.json"))

n_bellman_iterations = p["n_epochs"] * p["n_bellman_iterations_per_epoch"]
n_states_x = p["n_states_x"]
n_states_v = p["n_states_v"]
n_seeds = len(seeds)
ticks_freq = 3

### Extract data

In [ ]:
optimal_q = np.load(f"old_figures/data/optimal/Q.npy")
optimal_v = np.load(f"old_figures/data/optimal/V.npy")

ifqi_q = np.zeros((len(list_bellman_iterations_scope), n_seeds, n_bellman_iterations + 1, n_states_x, n_states_v, 2))
ifqi_v = np.zeros((len(list_bellman_iterations_scope), n_seeds, n_bellman_iterations + 1, n_states_x, n_states_v))
ifqi_ae = np.zeros((len(list_bellman_iterations_scope), n_seeds, n_bellman_iterations))
ifqi_be = np.zeros((len(list_bellman_iterations_scope), n_seeds, n_bellman_iterations + 1))

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    for idx_seed, seed in enumerate(seeds):
        ifqi_q[idx, idx_seed] = np.load(f"old_figures/{experiment_name}/iFQI/{bellman_iterations_scope}_Q_{seed}.npy")
        ifqi_v[idx, idx_seed] = np.load(f"old_figures/{experiment_name}/iFQI/{bellman_iterations_scope}_V_{seed}.npy")
        ifqi_ae[idx, idx_seed] = np.load(f"old_figures/{experiment_name}/iFQI/{bellman_iterations_scope}_A_{seed}.npy")
        ifqi_be[idx, idx_seed] = np.load(f"old_figures/{experiment_name}/iFQI/{bellman_iterations_scope}_B_{seed}.npy")

samples_mask = np.load(f"old_figures/{experiment_name}/samples_count.npy")
samples_mask_q_format = np.repeat(samples_mask[:, :, None], 2, axis=-1)

### Distances with the optimal Q function

In [ ]:
ifqi_q_mean = np.sqrt(np.mean(np.square(ifqi_q - optimal_q) * samples_mask_q_format, axis=(3, 4, 5))).mean(axis=1)
ifqi_q_std = np.sqrt(np.mean(np.square(ifqi_q - optimal_q) * samples_mask_q_format, axis=(3, 4, 5))).std(axis=1)
ifqi_q_confidence_interval = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations + 1))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_q_confidence_interval[idx] = confidence_interval(ifqi_q_mean[idx], ifqi_q_std[idx], n_seeds)

In [ ]:
import matplotlib.pyplot as plt 

colors = {
    "0": "#984ea3",
    "1": "#e41a1c",
    "2": "#4daf4a",
    "3": "#377eb8",
    "4": "#ff7f00",
    "5": "#f781bf",
    "6": "#a65628",
    "7": "#999999",
    "8": "#dede00",
}


plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

iterations = range(n_bellman_iterations + 1)

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    plt.plot(iterations, ifqi_q_mean[idx], label=f"K={bellman_iterations_scope}", color=colors[f"{idx}"], zorder=1)
    plt.scatter(iterations[::bellman_iterations_scope], ifqi_q_mean[idx, :: bellman_iterations_scope], marker="|", color=colors[f"{idx}"], zorder=1)
    plt.fill_between(iterations, ifqi_q_confidence_interval[idx, 0], ifqi_q_confidence_interval[idx, 1], color=colors[f"{idx}"], alpha=0.3)

plt.xticks(range(0, n_bellman_iterations + 1, ticks_freq))
plt.xlabel("#Iterations")
plt.title(r"$|| Q^* - Q_i ||_2$")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"oldfigures/{experiment_name}/distance_to_optimal_Q.pdf", bbox_inches='tight')

### Visualization of the policies at iteration max_bellman_iterations

In [ ]:
from experiments.car_on_hill.utils import TwoDimesionsMesh

max_pos = 1.0
max_velocity = 3.0

states_x = np.linspace(-max_pos, max_pos, n_states_x)
states_v = np.linspace(-max_velocity, max_velocity, n_states_v)

q_visu_mesh = TwoDimesionsMesh(states_x, states_v, axis_equal=False, zero_centered=True)

optimal_pi_weighted_average = (2 * ((optimal_q[:, :, 1] > optimal_q[:, :, 0])).astype(float) - 1) + (optimal_q[:, :, 1] == optimal_q[:, :, 0]).astype(float) / 2

q_visu_mesh.set_values(optimal_pi_weighted_average)
q_visu_mesh.show("$\pi^*$")
# _ = plt.savefig(f"figures/{experiment_name}/optimal_pi.pdf", bbox_inches='tight')

In [ ]:
for idx in range(len(list_bellman_iterations_scope)):
    ifqi_left_q = ifqi_q[idx, :, -1, :, :, 0]
    ifqi_right_q = ifqi_q[idx, :, -1, :, :, 1]
    ifqi_pi_weighted_average = (2 * (ifqi_right_q > ifqi_left_q).mean(axis=0) - 1)

    q_visu_mesh.set_values(ifqi_pi_weighted_average)
    q_visu_mesh.show(f"$\pi^{list_bellman_iterations_scope[idx]}$", xlabel="x", ylabel="v")
    # _ = plt.savefig(f"figures/{experiment_name}/ifqi_pi_{list_bellman_iterations_scope[idx]}.pdf", bbox_inches='tight')

### Distance to optimal value function

In [ ]:
ifqi_v_mean = np.sqrt(np.mean(np.square(ifqi_v - optimal_v) * samples_mask, axis=(3, 4))).mean(axis=1)
ifqi_v_std = np.sqrt(np.mean(np.square(ifqi_v - optimal_v) * samples_mask, axis=(3, 4))).std(axis=1)
ifqi_v_confidence_interval = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations + 1))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_v_confidence_interval[idx] = confidence_interval(ifqi_v_mean[idx], ifqi_v_std[idx], n_seeds)

In [ ]:
for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    if bellman_iterations_scope in [1, 5]:
        for k in [12, 14, 16, 18, 20]:
            gap = (ifqi_v_confidence_interval[idx, 1, k] - ifqi_v_confidence_interval[idx, 0, k]) / 2

            print("--- K=", bellman_iterations_scope, " ---")
            print(f"k={k}", np.around(ifqi_v_mean[idx][k], 2), "+-", np.around(gap, 2))
            print()

In [ ]:
import matplotlib.pyplot as plt 

colors = {
    "0": "#984ea3",
    "1": "#e41a1c",
    "2": "#4daf4a",
    "3": "#377eb8",
    "4": "#ff7f00",
    "5": "#f781bf",
    "6": "#a65628",
    "7": "#999999",
    "8": "#dede00",
}


plt.rc("font", size=22, family='serif', serif="Times New Roman")  # 18 for main paper and 15 for the table of figures
plt.rc("lines", linewidth=3)

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)

iterations = range(n_bellman_iterations + 1)

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    ax.plot(iterations, ifqi_v_mean[idx], label=f"K={bellman_iterations_scope}", color=colors[f"{idx}"], zorder=1)
    # plt.scatter(iterations[::bellman_iterations_scope], ifqi_v_mean[idx, :: bellman_iterations_scope], marker="|", color=colors[f"{idx}"], zorder=1)
    ax.fill_between(iterations, ifqi_v_confidence_interval[idx, 0], ifqi_v_confidence_interval[idx, 1], color=colors[f"{idx}"], alpha=0.3)

ax.set_xticks(range(0, n_bellman_iterations + 1, ticks_freq))
ax.set_xlabel(r"$k^{th}$ projection step")
ax.set_ylabel(r"$|| V^* - V^{\pi_k} ||_2$")
ax.set_title("Car-On-Hill")
ax.legend(ncols=2).set_zorder(3)
ax.grid(zorder=0)
# _ = fig.savefig(f"figures/distance_to_optimal_V.pdf", bbox_inches='tight')

### Value functions

In [ ]:
ifqi_v_mean_performance = np.mean(ifqi_v * samples_mask, axis=(3, 4)).mean(axis=1)
ifqi_v_std_performance = np.mean(ifqi_v * samples_mask, axis=(3, 4)).std(axis=1)
ifqi_v_confidence_interval_performance = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations + 1))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_v_confidence_interval_performance[idx] = confidence_interval(ifqi_v_mean_performance[idx], ifqi_v_std_performance[idx], n_seeds)

optimal_v_performance = (optimal_v * samples_mask).mean()

In [ ]:
import matplotlib.pyplot as plt 
from experiments import colors

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

iterations = range(n_bellman_iterations + 1)

plt.hlines(optimal_v_performance, 0, n_bellman_iterations, color="black", linestyle="--", label="Optimal")

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    plt.plot(iterations, ifqi_v_mean_performance[idx], label=f"K={bellman_iterations_scope}", color=colors[f"{idx}"], zorder=1)
    plt.scatter(iterations[::bellman_iterations_scope], ifqi_v_mean_performance[idx, :: bellman_iterations_scope], marker="|", color=colors[f"{idx}"], zorder=1)
    plt.fill_between(iterations, ifqi_v_confidence_interval_performance[idx, 0], ifqi_v_confidence_interval_performance[idx, 1], color=colors[f"{idx}"], alpha=0.3)

plt.xticks(range(0, n_bellman_iterations + 1, ticks_freq))
plt.xlabel("#Iterations")
plt.title(r"$|| J_i ||_2$")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"figures/{experiment_name}/J.pdf", bbox_inches='tight')

### Visualization of the last value functions

In [ ]:
from experiments.car_on_hill.utils import TwoDimesionsMesh

max_pos = 1.0
max_velocity = 3.0

states_x = np.linspace(-max_pos, max_pos, n_states_x)
states_v = np.linspace(-max_velocity, max_velocity, n_states_v)

q_visu_mesh = TwoDimesionsMesh(states_x, states_v)

q_visu_mesh.set_values(optimal_v * samples_mask)
q_visu_mesh.show(r"$V^{\pi^*}, \mathbb{E}\left[ V^{\pi^*} \right] =$" + str(np.round(np.mean(optimal_v * samples_mask), 2)))
# _ = plt.savefig(f"figures/{experiment_name}/optimal_V.pdf", bbox_inches='tight')

In [ ]:
for idx in range(len(list_bellman_iterations_scope)):
    ifqi_last_v = ifqi_v[idx, :, -1].mean(axis=0)

    q_visu_mesh.set_values(ifqi_last_v * samples_mask)
    q_visu_mesh.show(r"$V^{\pi_{ifqi}}, \mathbb{E}\left[ V^{\pi_{ifqi}} \right] =$" + f"{np.round(np.mean(ifqi_last_v * samples_mask), 4)}")
    # _ = plt.savefig(f"figures/{experiment_name}/ifqi_V_{list_bellman_iterations_scope[idx]}.pdf", bbox_inches='tight')

### Approximation error

In [ ]:
import jax.numpy as jnp

importance_iteration = jnp.ones(n_bellman_iterations)
importance_iteration = np.repeat(importance_iteration[None, :], n_seeds, axis=0)
importance_iteration = np.repeat(importance_iteration[None, :], len(list_bellman_iterations_scope), axis=0)

ifqi_ae_mean_performance = (ifqi_ae * importance_iteration).mean(axis=1)
ifqi_ae_std_performance = (ifqi_ae * importance_iteration).std(axis=1)
ifqi_ae_confidence_interval_performance = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_ae_confidence_interval_performance[idx] = confidence_interval(ifqi_ae_mean_performance[idx], ifqi_ae_std_performance[idx], n_seeds)

In [ ]:
import matplotlib.pyplot as plt 
from experiments import colors

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

iterations = range(1, n_bellman_iterations + 1)

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    plt.plot(iterations, ifqi_ae_mean_performance[idx], label=f"K={bellman_iterations_scope}", color=colors[f"{idx}"], zorder=1)
    plt.scatter(iterations[::bellman_iterations_scope], ifqi_ae_mean_performance[idx, :: bellman_iterations_scope], marker="|", color=colors[f"{idx}"], zorder=1)
    plt.fill_between(iterations, ifqi_ae_confidence_interval_performance[idx, 0], ifqi_ae_confidence_interval_performance[idx, 1], color=colors[f"{idx}"], alpha=0.3)

plt.xticks(range(1, n_bellman_iterations + 1, ticks_freq))
plt.xlabel("#Iterations")
plt.title("$|| \Gamma^*Q_{i - 1} - Q_i ||_2^2$")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"figures/{experiment_name}/ApproximationError.pdf", bbox_inches='tight')

In [ ]:
from idqn.utils.confidence_interval import importance_bound

importance_iteration = importance_bound(p["gamma"], n_bellman_iterations) * 0 + 1
importance_iteration = np.repeat(importance_iteration[None, :], n_seeds, axis=0)
importance_iteration = np.repeat(importance_iteration[None, :], len(list_bellman_iterations_scope), axis=0)

ifqi_ae_mean_performance = (ifqi_ae * importance_iteration).mean(axis=1)
ifqi_ae_std_performance = (ifqi_ae * importance_iteration).std(axis=1)
ifqi_ae_confidence_interval_performance = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_ae_confidence_interval_performance[idx] = confidence_interval(ifqi_ae_mean_performance[idx], ifqi_ae_std_performance[idx], n_seeds)

In [ ]:
import matplotlib.pyplot as plt 
from experiments import colors

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

for idx_bellman_iteration in range(n_bellman_iterations):
    plt.bar(range(len(list_bellman_iterations_scope)), ifqi_ae_mean_performance[:, idx_bellman_iteration], bottom=ifqi_ae_mean_performance[:, :idx_bellman_iteration].sum(axis=1), color=colors[f"{idx_bellman_iteration % len(colors)}"], zorder=2)

plt.title("$ \sum_{i=1}^N || \Gamma^*Q_{i - 1} - Q_i ||_2^2$")
plt.xticks(range(len(list_bellman_iterations_scope)), list_bellman_iterations_scope)
plt.grid(zorder=0)
plt.xlabel("k")
# _ = plt.savefig(f"figures/{experiment_name}/ApproximationError_bars.pdf", bbox_inches='tight')

### Bellman error

In [ ]:
ifqi_be_mean_performance = ifqi_be.mean(axis=1)
ifqi_be_std_performance = ifqi_be.std(axis=1)
ifqi_be_confidence_interval_performance = np.zeros((len(list_bellman_iterations_scope), 2, n_bellman_iterations + 1))

for idx in range(len(list_bellman_iterations_scope)):
    ifqi_be_confidence_interval_performance[idx] = confidence_interval(ifqi_be_mean_performance[idx], ifqi_be_std_performance[idx], n_seeds)

In [ ]:
import matplotlib.pyplot as plt 
from experiments import colors

plt.rc("font", size=15)
plt.rc("lines", linewidth=3)

iterations = range(1, n_bellman_iterations + 2)

for idx, bellman_iterations_scope in enumerate(list_bellman_iterations_scope):
    plt.plot(iterations, ifqi_be_mean_performance[idx], label=f"K={bellman_iterations_scope}", color=colors[f"{idx}"], zorder=1)
    # plt.scatter(iterations[::bellman_iterations_scope], ifqi_be_mean_performance[idx, :: bellman_iterations_scope], marker="|", color=colors[f"{idx}"], zorder=1)
    plt.fill_between(iterations, ifqi_be_confidence_interval_performance[idx, 0], ifqi_be_confidence_interval_performance[idx, 1], color=colors[f"{idx}"], alpha=0.3)

plt.xticks(range(1, n_bellman_iterations + 2, ticks_freq))
plt.xlabel("#Iterations")
plt.title("$|| \Gamma^*Q_{i-1} - Q_{i-1} ||_2^2$")
plt.legend().set_zorder(3)
plt.grid(zorder=0)
# _ = plt.savefig(f"figures/{experiment_name}/BellmanError.pdf", bbox_inches='tight')